In [ ]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import CATEGORY10, ColorScale , DateScale, LinearScale, Lines, Axis, Figure , OrdinalScale , Bars , Axis
from lib.ENTSOE import fetch_production
from lib.ENTSOE import fetch_price
import pandas as pd
from datetime import datetime, timedelta
import lib
from pkgutil import iter_modules
import importlib
import os
from ipywidgets import TwoByTwoLayout , Button, Layout, jslink, IntText, IntSlider, FloatSlider, Dropdown
import json
import random
import requests
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl
import ipywidgets as widgets
from ipyflex import FlexLayout


with open("./eu-countries.geo.json") as f:
    countries = json.load(f)
    
m = Map(zoom=3,center=(48.30, 23.23), zoom_control=False, layout=dict(height='100%', width='100%'))
m.add_control(ZoomControl(position='topright'))
m.add_control(FullScreenControl())
geo = GeoJSON(data=countries,style={"fillColor": "white", "weight": 0.5},hover_style={"fillColor": "#1f77b4"},name="Countries")
m.add_layer(geo)


data = fetch_production('IT')
param = []
labels = []


# Populate lists
productions = ["wind","coal","gas","nuclear","biomass","hydro","oil","solar","unknown"]
for production in productions:
    if production in data[0]['production'] and data[0]['production'][production] is not None:
        param.append([subdata['production'][production] for subdata in data])
        labels.append(production)

    
param2 = [param['datetime'] for param in data]
time_data = np.array(param2,dtype='datetime64')
fig = plt.figure(title="Italia", animation_duration=1000)
axes_options = {"x": {"label": "time "}, "y": {"label": "production (MWh) ","label_offset" : "6ex"}}
bar1 =  plt.bar(time_data,param,colors=CATEGORY10,labels=labels,display_legend=True)
axes = plt.axes(options = axes_options)
#display (fig)




#app = TwoByTwoLayout(bottom_right=k,bottom_left=m)
"""
widget_control1 = WidgetControl(widget=fig, position="bottomright")
m.add_control(widget_control1)
"""

def update_figure(param, labels , time_data , country):
    bar1.y = param
    bar1.x = time_data
    fig.title = country
    


def on_click(event, feature, **kwargs):
  
    data = fetch_production(feature['properties']['iso_a2'])
    data2 = fetch_price()
    param = []
    labels = []
    data = []
    productions = ["wind","coal","gas","nuclear","biomass","hydro","oil","solar","unknown"]
    for production in productions:
        if production in data[0]['production'] and data[0]['production'][production] is not None:
            param.append([subdata['production'][production] for subdata in data])
            labels.append(production)
    
    
    update_figure(param, labels , time_data ,feature['properties']['name'])

geo.on_click(on_click)

"""
dropdown = Dropdown(
    options=["data", "population", "lifeExpectancy"],
    value=data,
    description="Plotting:",
)


def on_click(change):
    global data_name

    data_name = change["new"]
    update_figure(param, labels , time_data ,feature['properties']['name'])


dropdown.observe(on_click, "value")

widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")

m.add_control(widget_control2)


    

clicked_country = None

def update_geojson(feature, **kwargs):
    global clicked_country
    data = fetch_production(feature['properties']['iso_a2'])
  
    if clicked_country is None:
        clicked_country = GeoJSON(data=data, style={"color": "blue","fillOpacity": 0.5})
        m.add_layer(clicked_country)
    else:
        clicked_country.data = data
    
     
    param = []
    labels = []
    productions = ["wind","coal","gas","nuclear","biomass","hydro","oil","solar","unknown"]
    for production in productions:
        if production in data[0]['production'] and data[0]['production'][production] is not None:
            param.append([subdata['production'][production] for subdata in data])
            labels.append(production)
    
    
    update_figure(param, labels , time_data ,feature['properties']['name'])

        
geo.on_click(update_geojson)
"""
m

In [ ]:
w = {'Map':m,'fig':fig}
header = dict(title='Electricity Production',
            style={'background':'green',
                    'color':'red',
                    'fontStyle': 'oblique'},
            )
FlexLayout(w,style = {'height':'2000px'},header=header,template = "Electricity_map.json")